## 📟 CodeSmell: The Passive-Aggressive Senior Dev Tutor

📟 CodeSmell: The Passive-Aggressive Senior Dev Tutor
CodeSmell is a real-time Python tutor designed for developers who find standard linters too polite. Powered by OpenAI’s Chat Completions API, this tool acts as a jaded, over-caffeinated Senior Developer who has no patience for your un-Pythonic habits.

Instead of dry error codes, CodeSmell delivers:
The Roast: A biting, snarky critique of your code's anti-patterns.
The Redemption: A streamed, optimized, and "Pythonic" rewrite of your logic.
Instant Feedback: Utilizing One-Shot Prompting and Streaming API calls for near-zero latency code reviews.

Stop writing code that makes your seniors cry. Submit your scripts, take the roast, and learn to write Python the way.

In [ ]:
# imports
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display

In [ ]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_GEMMA = 'gemma3:270m'
# Pull the model if not already pulled
!ollama list | grep -q "$MODEL_GEMMA" && echo "✅ $MODEL_GEMMA already exists" || (echo "🚀 Pulling $MODEL_GEMMA model..." && echo "⏳ This may take a few minutes depending on size..." && ollama pull "$MODEL_GEMMA" && echo "✅ Download completed! $MODEL_GEMMA is ready.")


In [ ]:
# set up environment
load_dotenv(override=True)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OLLAMA_API_KEY = os.getenv("OLLAMA_API_KEY")
OLLAMA_BASE_URL = os.getenv("OLLAMA_BASE_URL")


In [ ]:
# here is the question; type over this to ask something new

system_prompt = """
You are a brilliant but deeply impatient Senior Python Developer who has been reviewing junior code for 20 years. 
You hate inefficient loops, global variables, and poor naming conventions.
A junior is showing you their code. Roast the anti-pattern you see and provide a better way.
Response in a markdown format.
if the code is not python, respond politely to the user that you are only trained to answer Python related code questions.
Format your response as follows:
ROAST: [Your snarky comment]
FIX: [The Pythonic code]
"""

In [ ]:
openai = OpenAI()

def review_code(user_prompt,use_local_model=False):
    print(f"Using {MODEL_GPT if not use_local_model else MODEL_GEMMA} model")
    if use_local_model:
        openai = OpenAI(base_url=OLLAMA_BASE_URL,api_key=OLLAMA_API_KEY)
    else:
        openai = OpenAI()
    stream = openai.chat.completions.create(
        model=MODEL_GPT if not use_local_model else MODEL_GEMMA,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        stream=True,
    )
    print("--- LIVE CODE REVIEW ---")
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)
    print("\n-----------------------")

In [ ]:
python_code_prompt = """
Analyze the following Python code, identify the issues with it and provide a refactored version of the code.
def process_data(new_item, data_list=[]):
    data_list.append(new_item)
    f = open("log.txt", "a")
    report = ""
    for i in range(len(data_list)):
        report += "Item " + str(i) + ": " + str(data_list[i]) + "\n"
        
    f.write(report)
    f.close() 
    # (Note: if f.write fails, f.close never runs!)
    
    return data_list
"""

In [ ]:
non_python_code_prompt = """
Analyze the following code, identify the issues with it and provide a refactored version of the code.

using System;
using System.Collections.Generic;
using System.IO;
using System.Linq;

public class DataProcessor
{
    // Use 'null' as default for lists to avoid shared state issues
    public List<string> ProcessData(string newItem, List<string>? dataList = null)
    {
        // 1. Null Coalescing: Ensure we have a valid list instance
        dataList ??= new List<string>();
        dataList.Add(newItem);

        // 2. Resource Management: 'using' statement ensures the file is closed/disposed
        // even if an exception occurs during writing.
        using (StreamWriter sw = new StreamWriter("log.txt", append: true))
        {
            // 3. LINQ & Interpolation: Replaces manual for-loops and '+' concatenation.
            // Select maps the list to indexed strings; Join combines them efficiently.
            var reportLines = dataList
                .Select((item, index) => $"Item {index}: {item}");

            string finalReport = string.Join(Environment.NewLine, reportLines);
            
            sw.WriteLine(finalReport);
        }

        return dataList;
    }
}
"""

In [ ]:
# Get gpt-4o-mini to answer, with streaming
# you can try with the non_python_code_prompt or python_code_prompt to see how it works
review_code(python_code_prompt)

In [ ]:
# Get gemma3:270m to answer
# you can try with the non_python_code_prompt or python_code_prompt to see how it works
review_code(non_python_code_prompt, use_local_model=True)